In [1]:
import pandas as pd
import numpy as np

# Department 系所

In [2]:
dep = pd.read_csv('dataset/Department.csv')
dep['department_Cname_simple'] = ['商學院','國貿系','金融系','會計系','統計系','企管系','資管系','財管系','風管系','企研所(MBA學位學程)','科管智財所','營管碩','國營碩','產業博','外院'] # 後續資料整理用
dep

,department_id,department_Ename,department_Cname,department_Cname_simple
0,300,College of Commerce,商學院,商學院
1,301,Intl Buss,國際經營與貿易學系,國貿系
2,302,Finance,金融學系,金融系
3,303,Accounting,會計學系,會計系
4,304,Statistics,統計學系,統計系
5,305,BA,企業管理學系,企管系
6,306,MIS,資訊管理學系,資管系
7,307,M & B,財務管理學系,財管系
8,308,RMI,風險管理與保險學系,風管系
9,363,MBA,企業管理研究所(MBA),企研所(MBA學位學程)


# PartTime 校外兼職

In [3]:
parttime = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='112學年度商學院教師至校外兼職名冊') # 要刪除原始excel檔的第一列
parttime.rename(columns={'員工代號':'teacher_id', '兼職機關':'pt_company', '兼職單位':'pt_department', '兼職職稱':'pt_position','兼職起日':'pt_start','兼職迄日':'pt_end'}, inplace=True)
parttime.drop(columns=['序號','姓名','單位','職稱'], inplace=True)
parttime.head(3)

,teacher_id,pt_company,pt_department,pt_position,pt_start,pt_end
0,301018,美喆國際股份有限公司,NaN,獨立董事,110/08/12,113/08/11
1,301018,新盛力科技股份有限公司,NaN,獨立董事、薪資報酬委員暨審計委員,111/09/07,114/09/06
2,103902,行政院環境保護署,第4屆溫室氣體管理基金管理會,委員,111/06/01,113/05/31


In [6]:
# 民國年轉換為西元年
def pt_year(pt_original):
    temp = pt_original.split('/')
    temp[0] = str(int(temp[0])+1911)
    return '/'.join(temp)

parttime['pt_start'] = parttime['pt_start'].apply(lambda x: pt_year(x))
parttime['pt_end'] = parttime['pt_end'].apply(lambda x: pt_year(x))

print(parttime.shape)
parttime.head(3)

(196, 6)


,teacher_id,pt_company,pt_department,pt_position,pt_start,pt_end
0,301018,美喆國際股份有限公司,NaN,獨立董事,2021/08/12,2024/08/11
1,301018,新盛力科技股份有限公司,NaN,獨立董事、薪資報酬委員暨審計委員,2022/09/07,2025/09/06
2,103902,行政院環境保護署,第4屆溫室氣體管理基金管理會,委員,2022/06/01,2024/05/31


In [7]:
# 確認是否unique(是否有重複的row)
parttime.duplicated().sum()

0

In [93]:
# 匯出
parttime.to_csv('dataset/PartTime.csv', index=False)

# Teacher & FacultyType & Class

In [8]:
# 合併112學年度第一二學期開課資料
fullyear = pd.DataFrame()

# 112-1
temp = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='112-1商學院專兼任教師開課時數表')
temp.drop(columns=['序號','性別'], inplace=True)
temp.rename(columns={'姓名':'Cname', '員工編號':'teacher_id', '單位名稱':'department_Cname_simple', '教師任職':'teacher_type'}, inplace=True)
temp_colname = temp.columns.tolist()
temp_colname[5:] = ['學士班', 'MBA', 'IMBA', '一般碩士班', '博士班', 'EMBA', 'DBA']
temp.columns = temp_colname # 更改欄位名稱
temp['year'] = 112 # 紀錄資料所屬學年度
temp['semester'] = 1 # 紀錄資料所屬學期
fullyear = pd.concat([fullyear, temp], axis=0)

# 112-2
temp = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='112-2商學院專兼任教師開課時數表')
temp.drop(columns=['序號','性別'], inplace=True)
temp.rename(columns={'姓名':'Cname', '員工編號':'teacher_id', '單位名稱':'department_Cname_simple', '教師任職':'teacher_type'}, inplace=True)
temp_colname = temp.columns.tolist()
temp_colname[5:] = ['學士班', 'MBA', 'IMBA', '一般碩士班', '博士班', 'EMBA', 'DBA']
temp.columns = temp_colname # 更改欄位名稱
temp['year'] = 112 # 紀錄資料所屬學年度
temp['semester'] = 2 # 紀錄資料所屬學期
fullyear = pd.concat([fullyear, temp], axis=0)

print(fullyear.shape)
fullyear.head(3)

(533, 14)


,Cname,teacher_id,department_Cname_simple,teacher_type,職稱,學士班,MBA,IMBA,一般碩士班,博士班,EMBA,DBA,year,semester
0,于紀隆,114576,會計系,兼任,教授級約聘教學人員,0,0,0,0,0,0,0,112,1
1,王文英,101476,會計系,專任,教授,1,0,0,1,0,0,0,112,1
2,王正偉,308001,風管系,專任,講師,6,0,0,0,0,0,0,112,1


## Teacher 教師

In [9]:
teacher = fullyear[['teacher_id','Cname','職稱','department_Cname_simple']].copy() # 補：Ename, job_title, year, dep_id
teacher

,teacher_id,Cname,職稱,department_Cname_simple
0,114576,于紀隆,教授級約聘教學人員,會計系
1,101476,王文英,教授,會計系
2,308001,王正偉,講師,風管系
3,104280,王采彤,助教,財管系
4,101881,王淑珍,助教,會計系
...,...,...,...,...
250,107991,羅明琇,教授,企管系
251,128808,羅秉政,助理教授,金融系
252,102104,譚丹琪,教授,國貿系
253,122580,蘇瓜藤,教授,會計系


In [10]:
# job_title: 取代職稱並轉換成英文
eng_jt = {
    '教授': 'Professor',
    '副教授': 'Associate Professor',
    '助理教授': 'Assistant Professor',
    '講師': 'Instructor'
}
teacher['job_title'] = teacher['職稱'].str.replace('級約聘教學人員|級專業技術人員', '', regex=True).replace(eng_jt)

teacher

,teacher_id,Cname,職稱,department_Cname_simple,job_title
0,114576,于紀隆,教授級約聘教學人員,會計系,Professor
1,101476,王文英,教授,會計系,Professor
2,308001,王正偉,講師,風管系,Instructor
3,104280,王采彤,助教,財管系,助教
4,101881,王淑珍,助教,會計系,助教
...,...,...,...,...,...
250,107991,羅明琇,教授,企管系,Professor
251,128808,羅秉政,助理教授,金融系,Assistant Professor
252,102104,譚丹琪,教授,國貿系,Professor
253,122580,蘇瓜藤,教授,會計系,Professor


In [11]:
# dep_id: 從dep帶入
teacher = teacher.merge(dep[['department_id','department_Cname_simple']].rename(columns={'department_id':'dep_id'}), on='department_Cname_simple', how='left')
teacher.drop(columns=['職稱','department_Cname_simple'], inplace=True)

teacher

,teacher_id,Cname,job_title,dep_id
0,114576,于紀隆,Professor,303
1,101476,王文英,Professor,303
2,308001,王正偉,Instructor,308
3,104280,王采彤,助教,307
4,101881,王淑珍,助教,303
...,...,...,...,...
528,107991,羅明琇,Professor,305
529,128808,羅秉政,Assistant Professor,302
530,102104,譚丹琪,Professor,301
531,122580,蘇瓜藤,Professor,303


In [12]:
# Ename, year: 參考2021教師履歷資料(因目前商學院沒有同名同姓的老師，所以直接用姓名merge)
temp = pd.read_csv('raw_data/Teacher_temp.csv')
temp.rename(columns={'姓名(中文)':'Cname','姓名(英文)':'Ename','到職日':'year'}, inplace=True)
temp.drop_duplicates(['Cname'], inplace=True)
teacher = teacher.merge(temp[['Cname','Ename','year']], how='left', on='Cname')
teacher.drop_duplicates(subset=['teacher_id'], keep='last', inplace=True) # 因資料為112第一與第二學期名單，故上下學期可能會有重複值，重複值選擇保留第二學期的 job_title & dep_id
teacher = teacher[['teacher_id','Cname','Ename','job_title','year','dep_id']]
teacher

,teacher_id,Cname,Ename,job_title,year,dep_id
5,125929,王智弘,"Wang, Jason Chih-Hung",Associate Professor,2019.0,300
19,111283,何小台,"Ho, Chester S.",Professor,2010.0,933
47,129823,李鐘培,NaN,Professor,NaN,933
50,127335,汪志謙,"Wang, Chih-Chien",Associate Professor,2019.0,300
52,131347,車麗梅,NaN,Professor,NaN,303
...,...,...,...,...,...,...
528,107991,羅明琇,"Lo, Ming-Shiow",Professor,2008.0,305
529,128808,羅秉政,NaN,Assistant Professor,NaN,302
530,102104,譚丹琪,"Tan, Dan-Chi",Professor,2001.0,301
531,122580,蘇瓜藤,"Su, Robert K.",Professor,1992.0,303


In [99]:
# 若上述改成
# teacher.drop_duplicates(subset=['teacher_id','dep_id'], keep='last', inplace=True)
# 此時執行以下
# teacher[teacher['teacher_id'].isin(teacher['teacher_id'][teacher['teacher_id'].duplicated()].tolist())].sort_values(by='teacher_id')
# 會發現有老師轉換系所

In [13]:
# 將原始資料教師清單+2021教師履歷所取得的資料，合併到teacher中
temp2 = pd.read_csv('raw_data/Teacher_temp2.csv')
temp2.dropna(inplace=True)
add_id = set(temp2['teacher_id'])-set(teacher['teacher_id'])
teacher = pd.concat([teacher, temp2[temp2['teacher_id'].isin(add_id)]], axis=0)
teacher

,teacher_id,Cname,Ename,job_title,year,dep_id
5,125929,王智弘,"Wang, Jason Chih-Hung",Associate Professor,2019.0,300
19,111283,何小台,"Ho, Chester S.",Professor,2010.0,933
47,129823,李鐘培,NaN,Professor,NaN,933
50,127335,汪志謙,"Wang, Chih-Chien",Associate Professor,2019.0,300
52,131347,車麗梅,NaN,Professor,NaN,303
...,...,...,...,...,...,...
388,126856,黃冠華,"Huang, Sunny",Assistant Professor,2020.0,300
390,104856,黃台心,"Huang, Tai-Hsin",Professor,2005.0,302
404,305011,黃秉德,"Huang, Ping-Der",Associate Professor,1991.0,305
406,303507,黃金發,"Hwang, Jin-Fa",Associate Professor,1980.0,303


In [14]:
# 確認是否unique(是否有重複的row)
teacher['teacher_id'].duplicated().sum()

0

In [102]:
# 匯出
teacher.to_csv('dataset/Teacher.csv', index=False)

In [103]:
# 補充：2021教師履歷有但商學院教師名單沒有的教師（應為外院教師）
print(set(temp['Cname'])-set(teacher['Cname']))

{'周振鋒', '王文杰', '洪德欽', '冷則剛', '丁克華', '周德宇', '沈宗倫', '朱德芳', '沈錳坤', '林祖嘉', '朱芳妮', '吳文傑', '林日璇'}


## FacultyType 專兼別

In [15]:
fullyear['sum_class'] = fullyear[['學士班', 'MBA', 'IMBA', '一般碩士班', '博士班', 'EMBA', 'DBA']].sum(axis=1) # 有開課才有專兼別

facultytype = fullyear.loc[fullyear['sum_class']>0, ['teacher_id','year','semester','teacher_type']]
facultytype

,teacher_id,year,semester,teacher_type
1,101476,112,1,專任
2,308001,112,1,專任
6,101388,112,1,專任
7,111389,112,1,兼任
8,124500,112,1,名譽
...,...,...,...,...
250,107991,112,2,專任
251,128808,112,2,專任
252,102104,112,2,專任
253,122580,112,2,兼任


In [16]:
# 除了專任教師外，其餘專兼別皆改為兼任
print(facultytype['teacher_type'].drop_duplicates().tolist()) # 查看原本除了專任兼任外，還有哪些類別
facultytype['teacher_type'] = facultytype['teacher_type'].apply(lambda x: x if x=='專任' else '兼任')
facultytype

['專任', '兼任', '名譽', '函聘博', '講座', '合聘', '輔系', '函聘', '博士生兼任講師']


,teacher_id,year,semester,teacher_type
1,101476,112,1,專任
2,308001,112,1,專任
6,101388,112,1,專任
7,111389,112,1,兼任
8,124500,112,1,兼任
...,...,...,...,...
250,107991,112,2,專任
251,128808,112,2,專任
252,102104,112,2,專任
253,122580,112,2,兼任


In [17]:
# 確認是否unique(是否有重複的row)
facultytype[['teacher_id','year','semester']].duplicated().sum()

0

In [107]:
# 匯出
facultytype.to_csv('dataset/FacultyType.csv', index=False)

## Class 開課時數

In [18]:
# 根據不同學制轉換成長表格
classdf = fullyear.melt(id_vars=['teacher_id','year','semester'], value_vars=['學士班', '一般碩士班', '博士班', 'MBA', 'IMBA', 'EMBA', 'DBA'], var_name='degree', value_name='num_class')
classdf = classdf[classdf['num_class']>0] # 只儲存有開課的資料
classdf

,teacher_id,year,semester,degree,num_class
1,101476,112,1,學士班,1
2,308001,112,1,學士班,6
6,101388,112,1,學士班,2
7,111389,112,1,學士班,3
11,131349,112,1,學士班,1
...,...,...,...,...,...
3294,107924,112,1,DBA,3
3318,102700,112,1,DBA,3
3379,111578,112,1,DBA,3
3443,118330,112,1,DBA,3


In [19]:
# 確認是否unique(是否有重複的row)
classdf[['teacher_id','year','semester','degree']].duplicated().sum()

0

In [110]:
# 匯出
classdf.to_csv('dataset/Class.csv', index=False)

# List 學術、實務與教學貢獻清單

In [192]:
# 合併2021教師履歷資料與新資料
listdf = pd.DataFrame()

In [193]:
# 2021教師履歷資料
dep = pd.read_csv('dataset/Department.csv')
temp = pd.read_csv('raw_data/List_temp.csv')
temp = temp.merge(dep[['department_id','department_Ename']], how='left', on='department_Ename').sort_values(by='department_id', ascending=True) # 加入 department_id
temp['item_name'] = temp['item_name'].apply(lambda x: x.rstrip())
print(temp.shape)
temp.drop_duplicates(subset=['Cname','item_name','item_year','scholarship_type','equis'], keep='first', inplace=True, ignore_index=True) # 剔除重複資料
temp = temp.merge(teacher[['teacher_id','Cname']], how='left', on='Cname') # 加入老師id
temp.drop_duplicates(subset=['Cname','item_name','item_year','scholarship_type','equis'], keep='last', inplace=True, ignore_index=True) # 讓有兩個id的老師只留下最後的一個id
temp.drop(columns=['Cname','department_id','department_Ename'], inplace=True)
print(temp.shape)
temp.head(3)

(2729, 12)
(2707, 10)


,item_name,item_year,item_type,journal,journal_type,co_worker_in,co_worker_out,scholarship_type,equis,teacher_id
0,"柯冠成;蘇湘茹;林信助;朱香蕙*, 2016.06, '技術分析對台灣波動度及規模效果之影響...",2016.0,NaN,Journal of Management and Business Research (管...,O,NaN,柯冠成;蘇湘茹;朱香蕙,Basic_or_Discovery_Scholarship,E1:Academic Research Articles,102349.0
1,"Yen;Yu-Min Yen*, 2017.09, 'Risk Evaluations wi...",2017.0,NaN,Journal of Banking and Finance,O,NaN,Ray Yeutien Chou;Tso-Jung,Basic_or_Discovery_Scholarship,E1:Academic Research Articles,118586.0
2,公司理財整合型研究-子計畫三: 股東權益成本之探討:以指數成分股變動為例(2/3),2020.0,NaN,NaN,X,NaN,NaN,Basic_or_Discovery_Scholarship,E3:Studies and Reports commissioned by compani...,101607.0


In [194]:
# 2021教師履歷資料: 補部分 journal_type
temp['journal_type'].fillna('X')
temp.dropna(subset=['teacher_id','item_name','item_year'], inplace=True, ignore_index=True) # 這幾個欄位有na者移除

# 確認是否unique(是否有重複的row)
print(temp.duplicated(subset=['teacher_id','item_name','item_year','equis']).sum())
# temp.drop_duplicates(subset=['teacher_id','item_name','item_year','equis'], keep='last', inplace=True, ignore_index=True)
print(temp.shape)

0
(2575, 10)


In [195]:
listdf = pd.concat([listdf, temp[['teacher_id','item_name', 'item_year', 'journal', 'journal_type','co_worker_in', 'co_worker_out', 'scholarship_type', 'equis']]], ignore_index=True)
listdf['teacher_id'] = listdf['teacher_id'].astype(int)
listdf['item_year'] = listdf['item_year'].astype(int)
print(listdf.shape)
listdf.head(3)

(2575, 9)


,teacher_id,item_name,item_year,journal,journal_type,co_worker_in,co_worker_out,scholarship_type,equis
0,102349,"柯冠成;蘇湘茹;林信助;朱香蕙*, 2016.06, '技術分析對台灣波動度及規模效果之影響...",2016,Journal of Management and Business Research (管...,O,NaN,柯冠成;蘇湘茹;朱香蕙,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
1,118586,"Yen;Yu-Min Yen*, 2017.09, 'Risk Evaluations wi...",2017,Journal of Banking and Finance,O,NaN,Ray Yeutien Chou;Tso-Jung,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
2,101607,公司理財整合型研究-子計畫三: 股東權益成本之探討:以指數成分股變動為例(2/3),2020,NaN,X,NaN,NaN,Basic_or_Discovery_Scholarship,E3:Studies and Reports commissioned by compani...


## 論著資料

In [196]:
# 新資料：論著資料
temp = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='論著資料(2019~2023)')
temp.rename(columns={'教師編號':'teacher_id','論著名稱':'item_name','論著日期':'item_year','論著類別':'item_type','期刊名':'journal'}, inplace=True)
temp.drop(columns=['系所代碼','系所名稱','教師姓名','論著名稱二','審查制度','專書性質','著作人數'], inplace=True)
temp['teacher_id'] = temp['teacher_id'].astype(int)
temp['item_name'] = temp['item_name'].apply(lambda x: x.rstrip())
print(temp.shape)
temp.head(3)

(1270, 19)


,teacher_id,論著編號,item_name,item_year,item_type,journal,期刊資料庫所屬,合著者1,合著者2,合著者3,合著者4,合著者5,合著者6,合著者7,合著者8,合著者9,合著者10,合著者11,合著者12
0,102703,412899,Do local leads deliver contracting benefits? E...,2019,期刊論文,Asia-Pacific Journal of Accounting & Economics,SSCI,"Chow, Y. E.","Yoa, W. R.",Chen Lung Chin.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111520,416269,"Team goal orientation composition, team effica...",201911,期刊論文,Journal of Management & Organization,SSCI,"Huang, C. Y.","Huang, J. C.","Chang, Y. E.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119794,416411,由比較法觀點論會計師之責任限制,201906,期刊論文,證券市場發展季刊,TSSCI,陳俊元,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
# 新資料：論著資料-前處理

# item_year 只取年份
temp['item_year'] = temp['item_year'].apply(lambda x: int(str(x)[:4]))

# item_type 取代空格，不看學術交流，其他變成None
temp['item_type'] = temp['item_type'].str.replace(' ','')
temp = temp[temp['item_type']!='學術交流']
temp['item_type'] = temp['item_type'].apply(lambda x: None if x=='其他' else x)

# journal_type 先判斷三種
temp['journal_type'] = temp['期刊資料庫所屬'].apply(lambda x: 'O' if any(item in str(x).split() for item in ['SSCI','SCI','TSSCI']) else 'X')

# co_worker_in, co_worker_out 暫時先把所有合著者放在co_worker_out，之後由老師修改
temp['co_worker_in'] = None
temp['co_worker_out'] = temp.apply(lambda row: ';'.join([str(row[f'合著者{x}']) for x in range(1, 13) if pd.notna(row[f'合著者{x}'])]), axis=1)

# scholarship_type
temp['scholarship_type'] = np.where(temp['item_type']=='個案', 'Applied_or_Integration_Application_Scholarship', 'Basic_or_Discovery_Scholarship')

# equis
temp['equis'] = np.where(temp['item_type']=='個案', 'E6:Published Case Studies', 
                         np.where(temp['item_type']=='期刊論文', 'E1:Academic Research Articles',
                                  np.where(temp['item_type']=='會議論文','E4:Papers in Academic conferences',
                                           np.where(temp['item_type']=='專書','E7:Books (e.g. research monographs)',
                                                    np.where(temp['item_type']=='專書篇章','E8:Chapters in books',
                                                             np.where(temp['item_type']=='研究報告',"E12:Studies and Reports produced as part of an int'l network", None))))))

# 論著編號 刪除重複的值，不同年份視為不重複
temp.drop_duplicates(subset=['論著編號','item_year'], keep='last', inplace=True, ignore_index=True)

print(temp.shape)
temp.head(3)

(1220, 24)


,teacher_id,論著編號,item_name,item_year,item_type,journal,期刊資料庫所屬,合著者1,合著者2,合著者3,...,合著者8,合著者9,合著者10,合著者11,合著者12,journal_type,co_worker_in,co_worker_out,scholarship_type,equis
0,102703,412899,Do local leads deliver contracting benefits? E...,2019,期刊論文,Asia-Pacific Journal of Accounting & Economics,SSCI,"Chow, Y. E.","Yoa, W. R.",Chen Lung Chin.,...,NaN,NaN,NaN,NaN,NaN,O,None,"Chow, Y. E.;Yoa, W. R.;Chen Lung Chin.",Basic_or_Discovery_Scholarship,E1:Academic Research Articles
1,111520,416269,"Team goal orientation composition, team effica...",2019,期刊論文,Journal of Management & Organization,SSCI,"Huang, C. Y.","Huang, J. C.","Chang, Y. E.",...,NaN,NaN,NaN,NaN,NaN,O,None,"Huang, C. Y.;Huang, J. C.;Chang, Y. E.",Basic_or_Discovery_Scholarship,E1:Academic Research Articles
2,119794,416411,由比較法觀點論會計師之責任限制,2019,期刊論文,證券市場發展季刊,TSSCI,陳俊元,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,O,None,陳俊元,Basic_or_Discovery_Scholarship,E1:Academic Research Articles


In [198]:
# 新資料：論著資料-已有的item_name不再加入
mask = ~temp.set_index(['teacher_id','item_name','item_year']).index.isin(listdf.set_index(['teacher_id','item_name','item_year']).index)
temp = temp[mask].reset_index(drop=True)
print(temp.shape)

(928, 24)


In [199]:
# 確認是否unique(是否有重複的row)
# temp[temp.duplicated(subset=['teacher_id','item_name','item_year','equis'], keep=False)]
print(temp.duplicated(subset=['teacher_id','item_name','item_year','equis']).sum())
temp.drop_duplicates(subset=['teacher_id','item_name','item_year','equis'], keep='last', inplace=True, ignore_index=True)
print(temp.shape)

18
(910, 24)


In [200]:
listdf = pd.concat([listdf, temp[['teacher_id','item_name', 'item_year', 'journal', 'journal_type','co_worker_in', 'co_worker_out', 'scholarship_type', 'equis']]], ignore_index=True)
print(listdf.shape)
listdf.head(3)

(3485, 9)


,teacher_id,item_name,item_year,journal,journal_type,co_worker_in,co_worker_out,scholarship_type,equis
0,102349,"柯冠成;蘇湘茹;林信助;朱香蕙*, 2016.06, '技術分析對台灣波動度及規模效果之影響...",2016,Journal of Management and Business Research (管...,O,NaN,柯冠成;蘇湘茹;朱香蕙,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
1,118586,"Yen;Yu-Min Yen*, 2017.09, 'Risk Evaluations wi...",2017,Journal of Banking and Finance,O,NaN,Ray Yeutien Chou;Tso-Jung,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
2,101607,公司理財整合型研究-子計畫三: 股東權益成本之探討:以指數成分股變動為例(2/3),2020,NaN,X,NaN,NaN,Basic_or_Discovery_Scholarship,E3:Studies and Reports commissioned by compani...


## 研究計畫

In [201]:
# 新資料：研究計畫
temp = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='研究計畫(20190101~)')
temp.rename(columns={'教師代號':'teacher_id','計畫名稱':'item_name','起始年度':'item_year'}, inplace=True)
temp['teacher_id'] = temp['teacher_id'].astype(int)
temp['item_name'] = temp['item_name'].apply(lambda x: x.rstrip())
temp['journal'] = None
temp['journal_type'] = 'X'
temp['co_worker_out'] = None
temp.drop(columns=['系所代碼','系所名稱','計畫職稱','核定年度','主持人代碼','主持人','委託單位'], inplace=True) # '起迄日期'
print(temp.shape)
temp.head(3)

(925, 10)


,teacher_id,教師姓名,item_year,校內編號,item_name,計畫類別,起迄日期,journal,journal_type,co_worker_out
0,115045,湛可南,109,107B109001,資訊交易，共同基金，與公司財務(3/3),科技部,2020/08/01-2021/07/31,None,X,None
1,113615,周冠男,109,107B109002,定錨偏誤，報酬可預測性及公司決策(3/3),科技部,2020/08/01-2021/07/31,None,X,None
2,102106,戚務君,109,107B109003,審計對盈餘可比性的影響：法律責任、會計師特性與關鍵查核事項(3/3),科技部,2020/08/01-2021/07/31,None,X,None


In [202]:
# 新資料：研究計畫-前處理

# item_year 西元年
temp['item_year'] = temp['item_year'].apply(lambda x: int(x+1911))

# co_worker_in 同個計畫的內教師都先放在校內合著者
temp['co_worker_in'] = temp.groupby(['校內編號', 'item_name'])['教師姓名'].transform(lambda x: ';'.join(x))
temp['co_worker_in'] = temp.apply(lambda row: ';'.join([name for name in row['co_worker_in'].split(';') if name != row['教師姓名']]), axis=1) # 移除自己的姓名

# scholarship_type 科技部為basic，非科技部為applied
temp['scholarship_type'] = np.where(temp['計畫類別']=='科技部','Basic_or_Discovery_Scholarship','Applied_or_Integration_Application_Scholarship')

# equis 科技部為E15，非科技部為E3
temp['equis'] = np.where(temp['計畫類別']=='科技部','E15:Other(Please describe)其他(科技部學術型計畫 MOST Research Project)',"E3:Studies and Reports commissioned by companies and gov't agencies(企業、政府、科技部產學合作計畫)")

# item_name 處理相同 'teacher_id', 'item_year', 'item_name' 的 item_name -1, -2, -3...
def update_item_names(group):
    group = group.sort_values(by='起迄日期').reset_index(drop=True)
    if len(group)>1:
        group['item_name'] = [group['item_name'][0]+'-'+str(i + 1) for i in range(len(group))]
    return group
temp = temp.groupby(['teacher_id', 'item_year', 'item_name'], group_keys=False).apply(update_item_names)

print(temp.shape)
temp.head(3)

(925, 13)


,teacher_id,教師姓名,item_year,校內編號,item_name,計畫類別,起迄日期,journal,journal_type,co_worker_out,co_worker_in,scholarship_type,equis
0,101388,王儷玲,2019,108F108069,期貨公會活動補助案,非科技部,2019/10/01-2019/12/31,None,X,None,,Applied_or_Integration_Application_Scholarship,E3:Studies and Reports commissioned by compani...
0,101388,王儷玲,2019,108A108131,開放銀行在電信業之研究與應用,非科技部,2019/12/15-2020/12/31,None,X,None,陳恭;謝明華;彭金隆;宋皇志,Applied_or_Integration_Application_Scholarship,E3:Studies and Reports commissioned by compani...
0,101388,王儷玲,2020,109A109129,2020金融服務產業未來展望：數位科技創新,非科技部,2020/09/15-2021/03/31,None,X,None,謝明華;彭金隆,Applied_or_Integration_Application_Scholarship,E3:Studies and Reports commissioned by compani...


In [203]:
# 新資料：研究計畫-已有的item_name不再加入
mask = ~temp.set_index(['teacher_id','item_name','item_year']).index.isin(listdf.set_index(['teacher_id','item_name','item_year']).index)
temp = temp[mask].reset_index(drop=True)
print(temp.shape)

(787, 13)


In [204]:
# 確認是否unique(是否有重複的row)
# temp[temp.duplicated(subset=['teacher_id','item_name','item_year','equis'], keep=False)]
print(temp.duplicated(subset=['teacher_id','item_name','item_year','equis']).sum())
# temp.drop_duplicates(subset=['teacher_id','item_name','item_year','equis'], keep='last', inplace=True, ignore_index=True)
# print(temp.shape)

0


In [205]:
listdf = pd.concat([listdf, temp[['teacher_id','item_name', 'item_year', 'journal', 'journal_type','co_worker_in', 'co_worker_out', 'scholarship_type', 'equis']]], ignore_index=True)
print(listdf.shape)
listdf.head(3)

(4272, 9)


,teacher_id,item_name,item_year,journal,journal_type,co_worker_in,co_worker_out,scholarship_type,equis
0,102349,"柯冠成;蘇湘茹;林信助;朱香蕙*, 2016.06, '技術分析對台灣波動度及規模效果之影響...",2016,Journal of Management and Business Research (管...,O,NaN,柯冠成;蘇湘茹;朱香蕙,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
1,118586,"Yen;Yu-Min Yen*, 2017.09, 'Risk Evaluations wi...",2017,Journal of Banking and Finance,O,NaN,Ray Yeutien Chou;Tso-Jung,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
2,101607,公司理財整合型研究-子計畫三: 股東權益成本之探討:以指數成分股變動為例(2/3),2020,NaN,X,NaN,NaN,Basic_or_Discovery_Scholarship,E3:Studies and Reports commissioned by compani...


## 獎項

In [206]:
# 新資料：獎項
temp = pd.read_excel('raw_data/電算中心原始資料.xlsx', sheet_name='獎項(108學年~)')
temp.rename(columns={'教師代號':'teacher_id','年度':'item_year'}, inplace=True)
temp['teacher_id'] = temp['teacher_id'].astype(int)
temp['journal'] = None
temp['journal_type'] = 'X'
temp['co_worker_in'] = None
temp['co_worker_out'] = None
temp.drop(columns=['單位代碼','單位名稱','序號','教師姓名','頒獎單位'], inplace=True) # '起迄日期'
print(temp.shape)
temp.head(3)

(285, 8)


,teacher_id,item_year,獎項,備註,journal,journal_type,co_worker_in,co_worker_out
0,102104,108,科技部研究獎勵 ...,NaN,None,X,None,None
1,102104,108,科技部研究獎勵 ...,NaN,None,X,None,None
2,111580,108,科技部研究獎勵 ...,NaN,None,X,None,None


In [207]:
# 新資料：獎項-前處理  scholarship_type	equis

# item_year 西元年
temp['item_year'] = temp['item_year'].apply(lambda x: int(x+1911))

# item_name 加入備註
temp['備註'].fillna(0, inplace=True)
temp['item_name'] = temp.apply(lambda row: str(int(row['備註']))+row['獎項'] if row['備註']!=0 else row['獎項'], axis=1)
temp['item_name'] = temp['item_name'].apply(lambda x: x.rstrip())

# scholarship_type basic or applied，先算在basic，之後給老師調整
temp['scholarship_type'] = 'Basic_or_Discovery_Scholarship'

# equis E15
temp['equis'] = 'E15:Other(Competitive Research Awards Received)獲重要研究獎項次數'

# 移除重複值
temp.drop_duplicates(subset=['teacher_id','item_year','item_name'], inplace=True, ignore_index=True)

print(temp.shape)
temp.head(3)

(235, 11)


,teacher_id,item_year,獎項,備註,journal,journal_type,co_worker_in,co_worker_out,item_name,scholarship_type,equis
0,102104,2019,科技部研究獎勵 ...,0.0,None,X,None,None,科技部研究獎勵,Basic_or_Discovery_Scholarship,E15:Other(Competitive Research Awards Received...
1,111580,2019,科技部研究獎勵 ...,0.0,None,X,None,None,科技部研究獎勵,Basic_or_Discovery_Scholarship,E15:Other(Competitive Research Awards Received...
2,111580,2019,學術研究優良獎 ...,0.0,None,X,None,None,學術研究優良獎,Basic_or_Discovery_Scholarship,E15:Other(Competitive Research Awards Received...


In [208]:
# 新資料：獎項-已有的item_name不再加入
mask = ~temp.set_index(['teacher_id','item_name','item_year']).index.isin(listdf.set_index(['teacher_id','item_name','item_year']).index)
temp = temp[mask].reset_index(drop=True)
print(temp.shape)

(221, 11)


In [209]:
# 確認是否unique(是否有重複的row)
# temp[temp.duplicated(subset=['teacher_id','item_name','item_year','equis'], keep=False)]
print(temp.duplicated(subset=['teacher_id','item_name','item_year','equis']).sum())
# temp.drop_duplicates(subset=['teacher_id','item_name','item_year','equis'], keep='last', inplace=True, ignore_index=True)
# print(temp.shape)

0


In [210]:
listdf = pd.concat([listdf, temp[['teacher_id','item_name', 'item_year', 'journal', 'journal_type','co_worker_in', 'co_worker_out', 'scholarship_type', 'equis']]], ignore_index=True)
print(listdf.shape)
listdf.head(3)

(4493, 9)


,teacher_id,item_name,item_year,journal,journal_type,co_worker_in,co_worker_out,scholarship_type,equis
0,102349,"柯冠成;蘇湘茹;林信助;朱香蕙*, 2016.06, '技術分析對台灣波動度及規模效果之影響...",2016,Journal of Management and Business Research (管...,O,NaN,柯冠成;蘇湘茹;朱香蕙,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
1,118586,"Yen;Yu-Min Yen*, 2017.09, 'Risk Evaluations wi...",2017,Journal of Banking and Finance,O,NaN,Ray Yeutien Chou;Tso-Jung,Basic_or_Discovery_Scholarship,E1:Academic Research Articles
2,101607,公司理財整合型研究-子計畫三: 股東權益成本之探討:以指數成分股變動為例(2/3),2020,NaN,X,NaN,NaN,Basic_or_Discovery_Scholarship,E3:Studies and Reports commissioned by compani...


In [211]:
# 確認是否unique(是否有重複的row)
print(listdf.duplicated(subset=['teacher_id','item_name','item_year','equis']).sum())

0


In [218]:
# 匯出
listdf.dropna(subset=['teacher_id','item_name','item_year','scholarship_type','equis'], inplace=True, ignore_index=True)
print(listdf.shape)
listdf.to_csv('dataset/List1.csv', index=False)

(4471, 9)


# Resume 教師履歷
匯入歷年資料用，未來不用執行

In [125]:
dep = pd.read_csv('dataset/Department.csv')

temp = pd.read_csv('raw_data/Resume_temp.csv')
temp = temp.merge(dep[['department_id','department_Ename']], how='left', on='department_Ename') # 加入 department_id，稍後用來對照teacher有否該組(teacher_id,dep_id)資料
temp = temp[temp['teacher_id'].isin(teacher['teacher_id'].tolist())]
temp.sort_values(by=['teacher_id','department_id'], ascending=True, inplace=True, ignore_index=True)
print(temp.shape)
temp.head(3)

(374, 29)


,teacher_id,resume_year,department_Ename,highest_edu_degree,highest_edu_department,highest_edu_school,highest_edu_graduation_year,experience,teaching_interests,research_interests,...,PA1-6,PA1-7&IP1-3,PA1-8&SP1-3,SP1-1,SP1-2,IP1a,IP1-2,IP1-4,IP1-5,department_id
0,101388,2021,RMI,PhD,NaN,NaN,1998,"1. Chairperson,Pension Fund Association, 2011-...","Risk Management and Insurance, Pension Plan an...","Pension Plan, Insurance Finance and Accounting",...,0,0,1,1,1,0,0,0,0,308
1,101388,2021,EMBA,PhD,NaN,NaN,1998,"1. Chairperson,Pension Fund Association, 2011-...","Risk Management and Insurance, Pension Plan an...","Pension Plan, Insurance Finance and Accounting",...,0,0,1,1,1,0,0,0,0,932
2,101476,2021,Accounting,PhD,NaN,NaN,1998,"1. Associate Professor, Department of Accounti...","Cost and Managerial Accounting, Advanced Mana...","Cost and Managerial Accounting, Financial Acco...",...,1,0,1,1,0,0,0,0,0,303


In [126]:
# 原始教師履歷資料存在重複值，即一位教師會有不同系所（有至他系上課會納入計算），這邊選擇先刪除，只留主系所的資料
resume = pd.DataFrame()
# for id in temp['teacher_id'].drop_duplicates():
#     dep_num = teacher.loc[teacher['teacher_id']==id, 'dep_id'].values[0]
#     resume = pd.concat([resume, temp.loc[(temp['teacher_id']==id)&(temp['department_id']==dep_num),]], axis=0)
resume = temp.drop_duplicates(subset='teacher_id', keep='first', ignore_index=True).copy()
resume.drop(columns=['department_Ename','department_id'], inplace=True)
print(resume.shape)
resume.head(3)

(252, 27)


,teacher_id,resume_year,highest_edu_degree,highest_edu_department,highest_edu_school,highest_edu_graduation_year,experience,teaching_interests,research_interests,discipline,...,PA1-5,PA1-6,PA1-7&IP1-3,PA1-8&SP1-3,SP1-1,SP1-2,IP1a,IP1-2,IP1-4,IP1-5
0,101388,2021,PhD,NaN,NaN,1998,"1. Chairperson,Pension Fund Association, 2011-...","Risk Management and Insurance, Pension Plan an...","Pension Plan, Insurance Finance and Accounting",NaN,...,1,0,0,1,1,1,0,0,0,0
1,101476,2021,PhD,NaN,NaN,1998,"1. Associate Professor, Department of Accounti...","Cost and Managerial Accounting, Advanced Mana...","Cost and Managerial Accounting, Financial Acco...",NaN,...,1,1,0,1,1,0,0,0,0,0
2,101477,2021,PhD,NaN,NaN,1991,"1.Dean of International Cooperation, National ...","Human Resource Management, Social Innovation, ...","Human Resource Management, Intellectual Capita...",NaN,...,1,0,1,1,1,0,0,0,0,0


In [127]:
# 確認是否unique(是否有重複的row)
resume[['teacher_id','resume_year']].duplicated().sum()

0

In [128]:
# 匯出
resume.to_csv('dataset/Resume.csv', index=False)